In [0]:
# read the file from 'dbfs:/user/arundhuti/delta//member_churn/silver/' 
member_df = spark.read.format('delta').load('dbfs:/user/arundhuti/delta/member_churn/silver/')

display(member_df)

Transformaing the data

In [0]:
# ide

In [0]:
%python
from pyspark.sql import functions as F

#create age bucket, like 18-30, 31-45, 46-60, 61-75, 76-90
member_sil_df = member_df.withColumn(
    'Age_Bucket', 
    F.when(F.col('Age') <= 30, '18-30')
     .when(F.col('Age') <= 45, '31-45')
     .when(F.col('Age') <= 60, '46-60')
     .when(F.col('Age') <= 75, '61-75')
     .when(F.col('Age') <= 90, '76-90')
)
#display(df)

In [0]:
# # from pyspark.sql import functions as F

member_sil_df = member_sil_df.withColumn('Gender_modified', F.when(F.col('Gender') == 'Male', 1)
                              .when(F.col('Gender') == 'Female', 0)
                              .otherwise(None)
                            )


display(member_sil_df)

In [0]:
from pyspark.ml.feature import StringIndexer
gender_indexer = StringIndexer(inputCol='gender', outputCol='Gender_indexed', handleInvalid='skip')
member_sil_df = gender_indexer.fit(member_sil_df).transform(member_sil_df)

display(member_sil_df)

In [0]:
%python
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType

# Assemble the 'Total_Claim_Amount' into a vector column first
assembler = VectorAssembler(inputCols=['Total_Claim_Amount'], outputCol='Total_Claim_Amount_vec')
member_sil_df = assembler.transform(member_sil_df)

# Apply StandardScaler on the vector column
scaler = StandardScaler(inputCol='Total_Claim_Amount_vec', outputCol='Total_Claim_Amount_scaled', withMean=True, withStd=True)
scaler_model = scaler.fit(member_sil_df)
member_sil_df = scaler_model.transform(member_sil_df)

# Extract the scaled value from the vector to a double column
extract_value_udf = udf(lambda v: float(v[0]) if v is not None else None, DoubleType())
member_sil_df = member_sil_df.withColumn('Total_Claim_Amount_scaled', extract_value_udf(col('Total_Claim_Amount_scaled')))

# Drop the intermediate vector column
member_sil_df = member_sil_df.drop('Total_Claim_Amount_vec')

display(member_sil_df)

In [0]:
# from pyspark.sql import functions as F
# # get value from Total_Claim_Amount_scaled.values
# #create a loop which will go 1 by 1 on each row and get the value and stored it in 'val' and the iterate that with df
# for row in df.select('Total_Claim_Amount_scaled').collect():
#     val = row['Total_Claim_Amount_scaled'][0]
#     #print(val) 
#     df = df.withColumn('High_Risk_Flag', F.when(F.col('Total_Claim_Amount_scaled')[0] > 0.5, 1).otherwise(0))
# display(df)   

# # val = df.select('Total_Claim_Amount_scaled').collect()[0]['Total_Claim_Amount_scaled'][0]
# # # Create a new feature : High_Risk_Flag based on the condition, and use the 'val' value from above
# # df = df.withColumn('High_Risk_Flag', F.when(F.val > 0.5, 1).otherwise(0))
# # display(df)

# # from pyspark.sql import functions as F

# # # Extract the 'values' field from the UDT and then get the first item
# # df = df.withColumn(
# #     'High_Risk_Flag',
# #     F.when(F.col('Total_Claim_Amount_scaled').getField('values').getItem(0) > 0.5, 1).otherwise(0)
# # )
# # display(df)

In [0]:
from pyspark.sql.functions import when, col

# Add High_Risk_Flag column
member_sil_df = member_sil_df.withColumn(
    "High_Risk_Flag",
    when(
        (col("Chronic_Conditions") >= 2) |
        (col("Premium_Increase_Pct") >= 15) |
        (col("Late_Payments") >= 2) |
        (col("Customer_Service_Calls") >= 3),
        1
    ).otherwise(0)
)

# Optional: Display result
display(member_sil_df)


In [0]:
#write the data as delta format in the folder 'dbfs:/user/arundhuti/delta/featured_data/' in change data capture mode
member_sil_df.write.format('delta').mode('overwrite').option('mergeSchema', 'true').save('dbfs:/user/arundhuti/delta/member_churn/silver/')

In [0]:
%sql
--SELECT * FROM delta.`dbfs:/user/arundhuti/delta/member_churn/silver/`